In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.circuit.library import QFT

### Define components of the algorithm

In [2]:
def initialize_qubits(given_circuit, measurement_qubits, target_qubit):
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - START
    
    given_circuit.h(measurement_qubits)
    given_circuit.x(target_qubit)
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

In [3]:
def unitary_operator(given_circuit, control_qubit, target_qubit, theta):
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - START
    
    given_circuit.cu1(2*np.pi*theta, control_qubit, target_qubit)
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

In [4]:
def unitary_operator_exponent(given_circuit, control_qubit, target_qubit, theta, exponent):
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - START
    
    given_circuit.cu1(2*np.pi*theta*exponent, control_qubit, target_qubit)
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

In [5]:
def apply_iqft(given_circuit, measurement_qubits, n):
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - START
    
    given_circuit.append(QFT(n).inverse(), measurement_qubits)
    
    ### WRITE YOUR CODE BETWEEN THESE LINES - END

### Construct algorithm using components

In [6]:
def qpe_program(n, theta):
    
    # Create a quantum circuit on n+1 qubits (n measurement, 1 target)
    qc = QuantumCircuit(n+1, n)
    
    # Initialize the qubits
    initialize_qubits(qc, range(n), n)
    
    # Apply the controlled unitary operators in sequence
    for x in range(n):
        exponent = 2**(n-x-1)
        unitary_operator_exponent(qc, x, n, theta, exponent)
        
    # Apply the inverse quantum Fourier transform
    apply_iqft(qc, range(n), n)
    
    # Measure all qubits
    qc.measure(range(n), range(n))
  
    return qc

### Execute algorithm on simulator and real hardware with 4 qubits test case

In [ ]:
# Define number of qubits and theta value
n = 4; theta = 0.5
test_circuit = qpe_program(n, theta)
test_circuit.draw(output='text')

In [ ]:
from qiskit import Aer, execute
simulator = Aer.get_backend('qasm_simulator')
counts = execute(test_circuit, backend=simulator, shots=1000).result().get_counts(test_circuit)
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
import operator
highest_probability_outcome = max(counts.items(), key=operator.itemgetter(1))[0][::-1]
measured_theta = int(highest_probability_outcome, 2)/2**n
print("Using %d qubits with theta = %.2f, measured_theta = %.2f." % (n, theta, measured_theta))

In [ ]:
provider = IBMQ.load_account()
device = least_busy(provider.backends(simulator=False))
print("Running on current least busy device: ", device)

from qiskit.tools.monitor import job_monitor
job = execute(test_circuit, backend=device, shots=1000, max_credits=10)
job_monitor(job, interval = 2)

In [ ]:
counts = job.result().get_counts()
plot_histogram(counts)

## Benchmarking runtime for simulator

In [10]:
def p_even(lst):
    return [x for x in lst if not x % 2]

In [36]:
theta = 0.5 

for n in p_even(range(2, 26)):
    test_circuit = qpe_program(n, theta)
    
    # Import and define simulator
    from qiskit import Aer, execute
    simulator = Aer.get_backend('qasm_simulator')
    
    
    for i in range(5): 
        
        # Time the simulation runtime and store values
        benchmark_times = []
        
        import time
        start_time = time.time()

        counts = execute(test_circuit, backend=simulator, shots=1000).result().get_counts(test_circuit)

        benchmark_times.append(time.time() - start_time)
    
        print(str(n) + 'qubit times = ' + str(benchmark_times))

2qubit times = [0.009991645812988281]
2qubit times = [0.01399374008178711]
2qubit times = [0.01099085807800293]
2qubit times = [0.010996103286743164]
2qubit times = [0.010993242263793945]
4qubit times = [0.01898956298828125]
4qubit times = [0.014992237091064453]
4qubit times = [0.016986608505249023]
4qubit times = [0.012993812561035156]
4qubit times = [0.010995864868164062]
6qubit times = [0.014987945556640625]
6qubit times = [0.01554107666015625]
6qubit times = [0.015723705291748047]
6qubit times = [0.017856836318969727]
6qubit times = [0.014995098114013672]
8qubit times = [0.02198481559753418]
8qubit times = [0.019986867904663086]
8qubit times = [0.018990755081176758]
8qubit times = [0.018985271453857422]
8qubit times = [0.02298760414123535]
10qubit times = [0.024981975555419922]
10qubit times = [0.023984909057617188]
10qubit times = [0.024077415466308594]
10qubit times = [0.025217533111572266]
10qubit times = [0.03597712516784668]
12qubit times = [0.03497624397277832]
12qubit times 